In [3]:
# CFM 301 W23
# Final Project
# Keshav, Andrew, Gavin, Anson

import pandas as pd
import numpy as np
from pandasql import sqldf
import statsmodels.api as sm
import seaborn as sns
from scipy import stats
from sklearn import linear_model
import math

# Note the the code from the ML lectures provided was utilized here in order to build our model

# Python Kernal Version: 3.11.1

In [4]:
# Code as provided in ML lecture exercise 1

#dataset = pd.read_sas("dataset_files/provided_dataset_merged_df.sas7bdat", encoding="ISO-8859-1")
dataset = pd.read_csv("dataset_files/provided_dataset_merged_df.csv")
dataset = dataset[dataset['yyyymm'] >= 198401]
dataset['permno'] = dataset['permno'].astype(np.int64)
factors = list(dataset.columns[9:59]) + list(dataset.columns[81:])
ret_cols = list(dataset.columns[61:75])

#Filtering stocks based on stock price and market cap
dataset['PRC'] = abs(dataset['PRC'])
dataset['size'] = dataset['PRC'] * dataset['SHROUT'] * 1000
dataset['botm_size'] = dataset.groupby(["permno"])["size"].shift(1)
dataset['botm_prc'] = dataset.groupby(["permno"])["PRC"].shift(1)
dataset['permno_year'] = dataset['permno'].astype(str) + '_' + dataset['yyyymm'].astype(str).str.slice(0,4)

dates = np.sort(pd.unique(dataset["yyyymm"]))
januaries = list(filter(lambda x: x % 100 == 1, dates))

indices_to_drop = set()

for date in januaries:
  df = dataset.loc[dataset["yyyymm"] == date]
  for index, row in df.iterrows():
    if row["botm_prc"] < 5 or row['botm_size'] < 10 ** 8:
      indices_to_drop.add(row["permno_year"])
indices_to_drop
dataset = dataset[~dataset['permno_year'].isin(indices_to_drop)].reset_index(drop=True)
dataset

,permno,yyyymm,monthid,ticker,conm,gvkey,cusip,naics,gsubind,IM,...,STOCK_SPLIT,DIVAMT,SPREAD,O_SCORE_Q,HIRING_RATE,INVESTMENT_GROWTH_3Y,size,botm_size,botm_prc,permno_year
0,10026,198603.0,75.0,JJSF,J & J SNACK FOODS CORP,12825,466032109,311812,30202030,-0.183465,...,1.000000,NaN,1.5,NaN,NaN,NaN,4.133125e+07,NaN,NaN,10026_1986
1,10026,198604.0,76.0,JJSF,J & J SNACK FOODS CORP,12825,466032109,311812,30202030,0.636488,...,1.000000,NaN,1.5,-1.288174,NaN,NaN,4.424875e+07,4.133125e+07,21.250000,10026_1986
2,10026,198605.0,77.0,JJSF,J & J SNACK FOODS CORP,12825,466032109,311812,30202030,NaN,...,1.000000,NaN,NaN,-1.288174,NaN,NaN,6.224000e+07,4.424875e+07,22.750000,10026_1986
3,10026,198606.0,78.0,JJSF,J & J SNACK FOODS CORP,12825,466032109,311812,30202030,NaN,...,0.666667,0.00,NaN,-1.288174,NaN,NaN,5.252400e+07,6.224000e+07,32.000000,10026_1986
4,10026,198607.0,79.0,JJSF,J & J SNACK FOODS CORP,12825,466032109,311812,30202030,NaN,...,1.000000,NaN,NaN,-1.273626,NaN,NaN,3.282750e+07,5.252400e+07,18.000000,10026_1986
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
352443,93429,201802.0,458.0,CBOE,CBOE GLOBAL MARKETS INC,184500,12503M108,523210,40203040,0.145937,...,1.000000,NaN,NaN,-3.977201,0.0,NaN,1.262409e+10,1.515126e+10,134.389999,93429_2018
352444,93429,201803.0,459.0,CBOE,CBOE GLOBAL MARKETS INC,184500,12503M108,523210,40203040,0.193968,...,1.000000,0.27,NaN,-3.977201,0.0,NaN,1.284230e+10,1.262409e+10,112.010002,93429_2018
352445,93429,201804.0,460.0,CBOE,CBOE GLOBAL MARKETS INC,184500,12503M108,523210,40203040,0.175603,...,1.000000,NaN,NaN,-4.053557,0.0,NaN,1.201168e+10,1.284230e+10,114.099998,93429_2018
352446,93429,201805.0,461.0,CBOE,CBOE GLOBAL MARKETS INC,184500,12503M108,523210,40203040,0.081776,...,1.000000,0.27,NaN,-4.053557,0.0,NaN,1.097452e+10,1.201168e+10,106.779999,93429_2018


In [5]:
# Keshav: Imputing data as provided the code for in ML lecture exercise 1

# NEW IMPUTE METHOD
dataset.dropna(subset= ['ticker', 'conm', 'gvkey', 'cusip', 'naics', 'gsubind'], inplace = True)

# .groupby crates a Pandas groupby object, where I've grouped the rows by their permno
#grouped_med = merged_df.groupby(by= 'permno')
grouped_med = dataset.groupby(by= 'monthid')
# the lambda function gets the median per group in the groupby object, and fills the NaN values with the median per group
imputed_grouped = grouped_med.transform(lambda y: y.fillna(y.median()))

# This line assigns the values of the medians 
dataset = dataset.assign(**imputed_grouped.to_dict(orient='series'))

# Some NAN VALUES STILL IN DATA, DROPNA TO TAKE THEM OUT  - Figuring out what's going on with Python difficult.
# Suggest using other programs such as SAS if issue is severe
dataset.dropna(subset=dataset.columns.difference(['noOfMAActivity','STOCK_SPLIT','DIVAMT','SPREAD','O_SCORE_Q','HIRING_RATE','INVESTMENT_GROWTH_3Y']), inplace = True)

C:\Users\Andrew\AppData\Local\Temp/ipykernel_35160/326238453.py:10: FutureWarning: Dropping invalid columns in DataFrameGroupBy.transform is deprecated. In a future version, a TypeError will be raised. Before calling .transform, select only columns which should be valid for the transforming function.
  imputed_grouped = grouped_med.transform(lambda y: y.fillna(y.median()))
C:\Users\Andrew\AppData\Local\Temp/ipykernel_35160/326238453.py:10: FutureWarning: Dropping invalid columns in DataFrameGroupBy.transform is deprecated. In a future version, a TypeError will be raised. Before calling .transform, select only columns which should be valid for the transforming function.
  imputed_grouped = grouped_med.transform(lambda y: y.fillna(y.median()))
C:\Users\Andrew\AppData\Local\Temp/ipykernel_35160/326238453.py:10: FutureWarning: Dropping invalid columns in DataFrameGroupBy.transform is deprecated. In a future version, a TypeError will be raised. Before calling .transform, select only columns

In [6]:
dataset

,permno,yyyymm,monthid,ticker,conm,gvkey,cusip,naics,gsubind,IM,...,STOCK_SPLIT,DIVAMT,SPREAD,O_SCORE_Q,HIRING_RATE,INVESTMENT_GROWTH_3Y,size,botm_size,botm_prc,permno_year
0,10026,198603.0,75.0,JJSF,J & J SNACK FOODS CORP,12825,466032109,311812,30202030,-0.183465,...,1.000000,0.260,1.500,-3.187230,0.015939,0.298857,4.133125e+07,7.992216e+08,39.000000,10026_1986
1,10026,198604.0,76.0,JJSF,J & J SNACK FOODS CORP,12825,466032109,311812,30202030,0.636488,...,1.000000,0.230,1.500,-1.288174,0.016734,0.304601,4.424875e+07,4.133125e+07,21.250000,10026_1986
2,10026,198605.0,77.0,JJSF,J & J SNACK FOODS CORP,12825,466032109,311812,30202030,0.354652,...,1.000000,0.250,0.375,-1.288174,0.016734,0.304601,6.224000e+07,4.424875e+07,22.750000,10026_1986
3,10026,198606.0,78.0,JJSF,J & J SNACK FOODS CORP,12825,466032109,311812,30202030,0.308972,...,0.666667,0.000,0.500,-1.288174,0.018679,0.304601,5.252400e+07,6.224000e+07,32.000000,10026_1986
4,10026,198607.0,79.0,JJSF,J & J SNACK FOODS CORP,12825,466032109,311812,30202030,0.273834,...,1.000000,0.185,0.750,-1.273626,0.018679,0.304601,3.282750e+07,5.252400e+07,18.000000,10026_1986
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
352443,93429,201802.0,458.0,CBOE,CBOE GLOBAL MARKETS INC,184500,12503M108,523210,40203040,0.145937,...,1.000000,0.240,NaN,-3.977201,0.000000,0.129318,1.262409e+10,1.515126e+10,134.389999,93429_2018
352444,93429,201803.0,459.0,CBOE,CBOE GLOBAL MARKETS INC,184500,12503M108,523210,40203040,0.193968,...,1.000000,0.270,NaN,-3.977201,0.000000,0.129318,1.284230e+10,1.262409e+10,112.010002,93429_2018
352445,93429,201804.0,460.0,CBOE,CBOE GLOBAL MARKETS INC,184500,12503M108,523210,40203040,0.175603,...,1.000000,0.250,NaN,-4.053557,0.000000,0.129453,1.201168e+10,1.284230e+10,114.099998,93429_2018
352446,93429,201805.0,461.0,CBOE,CBOE GLOBAL MARKETS INC,184500,12503M108,523210,40203040,0.081776,...,1.000000,0.270,NaN,-4.053557,0.000000,0.129453,1.097452e+10,1.201168e+10,106.779999,93429_2018


In [7]:
# Keshav: TODO: NOTE THE DATA SHOULD BE WINSORIZED

# dataset['monthOfYear'] = dataset['monthid'] % 12
# print(dataset[['yyyymm', 'monthid', 'monthOfYear']])
# dataset.groupby(by= 'monthOfYear')


In [8]:
# Data Cleaning/Factor Selection
# 1. Reduce stocks based on stock price/market cap
# 2. Imputing
# 3. Winsorizing
# 4. Correlation matrix
# 5. Normalization
# 6. Split into training/test/validation sets
# 7. Fama-Macbeth regression on test data --> determine which factors to finalize/keep based on t-stats

In [9]:
# Out of Sample Testing
# 1. Get predicted returns based on alphas/factor betas (for Fama-Macbeth method)
# 2. ML Method

In [10]:
#Winsorizing the factors by month
for monthid, group in dataset.groupby('monthid'):
    for factor in factors:
        max_cutoff = group[factor].quantile(0.01)
        min_cutoff = group[factor].quantile(0.99)
        group[factor] = group[factor].clip(min_cutoff, max_cutoff)
        dataset.loc[dataset['monthid'] == monthid, factor] = group[factor].values
dataset[factors].describe()

C:\Users\Andrew\AppData\Roaming\Python\Python310\site-packages\pandas\core\generic.py:7246: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result[mask] = np.nan


,IM,range_20,log_vol_dollar_20,range_120,log_vol_dollar_120,xret_5,xret_10,xret_20,xret_indsize_20,xret_indsize_std20,...,Cto,pe_ttm,lag_log_size,noOfMAActivity,STOCK_SPLIT,DIVAMT,SPREAD,O_SCORE_Q,HIRING_RATE,INVESTMENT_GROWTH_3Y
count,351999.000000,351999.000000,351999.000000,351999.000000,351999.000000,351999.000000,351999.000000,351999.000000,351999.000000,351999.000000,...,351999.000000,351999.000000,351999.000000,268998.000000,351999.000000,351999.000000,133110.000000,351999.000000,351999.000000,351999.000000
mean,0.065611,0.028636,15.975694,0.028660,15.956880,0.002467,0.003260,0.005625,0.005601,0.017814,...,0.259333,42.307020,7.621082,1.433605,0.998768,0.197563,0.793336,-3.467232,0.063205,0.631278
std,0.176826,0.015972,2.125747,0.014143,2.105021,0.042811,0.059097,0.084047,0.080996,0.010339,...,0.197892,58.447260,1.656164,1.544565,0.021608,0.094981,0.614191,1.036873,0.179206,1.689755
min,-0.855087,0.003822,7.994402,0.005321,8.012983,-0.305643,-0.421193,-0.612410,-0.592648,0.003682,...,0.005305,2.574015,1.727095,1.000000,0.500000,0.000000,0.015000,-6.396240,-0.444543,-0.923538
25%,-0.026194,0.018370,14.627335,0.019248,14.602087,-0.018809,-0.027353,-0.039806,-0.037240,0.010722,...,0.115523,13.637003,6.429216,1.000000,1.000000,0.150000,0.400000,-3.959708,0.000000,-0.025267
50%,0.065426,0.024616,16.078717,0.025142,16.051366,0.000913,0.001740,0.003783,0.003688,0.015260,...,0.225587,19.036464,7.469657,1.000000,1.000000,0.175000,0.625000,-3.462700,0.020408,0.268585
75%,0.146003,0.034306,17.511090,0.034222,17.474671,0.022316,0.032798,0.049300,0.047048,0.022105,...,0.347884,30.988852,8.676401,1.500000,1.000000,0.230000,1.000000,-3.044801,0.080018,0.605908
max,2.318851,0.229597,20.569824,0.147269,20.423303,0.492096,0.490022,0.684095,0.649780,0.135778,...,1.237098,200.000000,12.517654,24.000000,1.000000,2.516000,4.000000,0.502599,2.216023,30.467908


In [11]:
ff3_monthly_df_original = pd.read_excel('dataset_files/ff3_monthly.xlsx', sheet_name='ff3_monthly')

In [12]:
ff3_monthly_df = ff3_monthly_df_original.copy()
ff3_monthly_df['monthid'] = ff3_monthly_df['month_id'] = (ff3_monthly_df['DATE'].dt.year - 1980) * 12 + ff3_monthly_df['DATE'].dt.month

dataset = pd.merge(dataset, ff3_monthly_df, how='left', on='monthid').reset_index(drop=True)
dataset

,permno,yyyymm,monthid,ticker,conm,gvkey,cusip,naics,gsubind,IM,...,botm_size,botm_prc,permno_year,RETX,SMB,HML,RF,MOM,DATE,month_id
0,10026,198603.0,75.0,JJSF,J & J SNACK FOODS CORP,12825,466032109,311812,30202030,-0.183465,...,7.992216e+08,39.000000,10026_1986,0.0488,-0.0059,-0.0039,0.0060,0.0241,1986-03-31,75
1,10026,198604.0,76.0,JJSF,J & J SNACK FOODS CORP,12825,466032109,311812,30202030,0.636488,...,4.133125e+07,21.250000,10026_1986,-0.0131,0.0278,-0.0287,0.0052,-0.0037,1986-04-30,76
2,10026,198605.0,77.0,JJSF,J & J SNACK FOODS CORP,12825,466032109,311812,30202030,0.354652,...,4.424875e+07,22.750000,10026_1986,0.0462,-0.0135,-0.0021,0.0049,0.0205,1986-05-30,77
3,10026,198606.0,78.0,JJSF,J & J SNACK FOODS CORP,12825,466032109,311812,30202030,0.308972,...,6.224000e+07,32.000000,10026_1986,0.0103,-0.0096,0.0128,0.0052,0.0507,1986-06-30,78
4,10026,198607.0,79.0,JJSF,J & J SNACK FOODS CORP,12825,466032109,311812,30202030,0.273834,...,5.252400e+07,18.000000,10026_1986,-0.0645,-0.0336,0.0470,0.0052,0.0183,1986-07-31,79
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
351994,93429,201802.0,458.0,CBOE,CBOE GLOBAL MARKETS INC,184500,12503M108,523210,40203040,0.145937,...,1.515126e+10,134.389999,93429_2018,-0.0365,0.0023,-0.0107,0.0011,0.0358,2018-02-28,458
351995,93429,201803.0,459.0,CBOE,CBOE GLOBAL MARKETS INC,184500,12503M108,523210,40203040,0.193968,...,1.262409e+10,112.010002,93429_2018,-0.0235,0.0405,-0.0023,0.0011,-0.0113,2018-03-29,459
351996,93429,201804.0,460.0,CBOE,CBOE GLOBAL MARKETS INC,184500,12503M108,523210,40203040,0.175603,...,1.284230e+10,114.099998,93429_2018,0.0028,0.0114,0.0054,0.0014,0.0036,2018-04-30,460
351997,93429,201805.0,461.0,CBOE,CBOE GLOBAL MARKETS INC,184500,12503M108,523210,40203040,0.081776,...,1.201168e+10,106.779999,93429_2018,0.0265,0.0526,-0.0318,0.0014,0.0389,2018-05-31,461


In [14]:
#Chosen portfolio factors
#portfolio_factors = ['IV_capm', 'beta_5y', 'lag_log_size', 'IM', 'xret_10', 'dp', 'BL', 'roe', 'BM', 'pe_ttm', 'O_SCORE_Q', 'HIRING_RATE', 'INVESTMENT_GROWTH_3Y', 'STOCK_SPLIT']
portfolio_factors = ['lag_log_size', 'roe', 'O_SCORE_Q', 'pe_ttm', 'INVESTMENT_GROWTH_3Y', 'RSI_20', 'ni_g_q']

# Code as provided in ML lecture exercise 1

pd.set_option("mode.chained_assignment", None)
corr_matrix = dataset[portfolio_factors].corr().abs()

# creates an upper triangular matrix of the corr matrix (since it's symmetric about the diagonal)
upper_tri = corr_matrix.where(np.triu(np.ones(corr_matrix.shape),k=1).astype(bool))

# gets the columns that fit the criteria to be dropped
to_drop = [column for column in upper_tri.columns if any(upper_tri[column] > 0.7)]
print(to_drop)
for factor in to_drop:
    portfolio_factors.remove(factor)

# drops those columns
dataset.drop(dataset[to_drop], axis=1, inplace= True)

[]


In [15]:
#Normalize by calculating z-scores for each chosen factor
for monthid, group in dataset.groupby('monthid'):
    zscore_cols = []
    for factor in portfolio_factors:
        #group[factor + '_zscore'] = group[factor]
        #group[factor + '_zscore'] = group[factor + '_zscore'].apply(lambda x: (x - group[factor].mean()) / group[factor].std())
        #zscore_cols.append(factor + '_zscore')
        group[factor] = group[factor].apply(lambda x: (x - group[factor].mean()) / group[factor].std())
        zscore_cols.append(factor)
    dataset.loc[dataset['monthid'] == monthid, zscore_cols] = group[zscore_cols].values

dataset

,permno,yyyymm,monthid,ticker,conm,gvkey,cusip,naics,gsubind,IM,...,botm_size,botm_prc,permno_year,RETX,SMB,HML,RF,MOM,DATE,month_id
0,10026,198603.0,75.0,JJSF,J & J SNACK FOODS CORP,12825,466032109,311812,30202030,-0.183465,...,7.992216e+08,39.000000,10026_1986,0.0488,-0.0059,-0.0039,0.0060,0.0241,1986-03-31,75
1,10026,198604.0,76.0,JJSF,J & J SNACK FOODS CORP,12825,466032109,311812,30202030,0.636488,...,4.133125e+07,21.250000,10026_1986,-0.0131,0.0278,-0.0287,0.0052,-0.0037,1986-04-30,76
2,10026,198605.0,77.0,JJSF,J & J SNACK FOODS CORP,12825,466032109,311812,30202030,0.354652,...,4.424875e+07,22.750000,10026_1986,0.0462,-0.0135,-0.0021,0.0049,0.0205,1986-05-30,77
3,10026,198606.0,78.0,JJSF,J & J SNACK FOODS CORP,12825,466032109,311812,30202030,0.308972,...,6.224000e+07,32.000000,10026_1986,0.0103,-0.0096,0.0128,0.0052,0.0507,1986-06-30,78
4,10026,198607.0,79.0,JJSF,J & J SNACK FOODS CORP,12825,466032109,311812,30202030,0.273834,...,5.252400e+07,18.000000,10026_1986,-0.0645,-0.0336,0.0470,0.0052,0.0183,1986-07-31,79
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
351994,93429,201802.0,458.0,CBOE,CBOE GLOBAL MARKETS INC,184500,12503M108,523210,40203040,0.145937,...,1.515126e+10,134.389999,93429_2018,-0.0365,0.0023,-0.0107,0.0011,0.0358,2018-02-28,458
351995,93429,201803.0,459.0,CBOE,CBOE GLOBAL MARKETS INC,184500,12503M108,523210,40203040,0.193968,...,1.262409e+10,112.010002,93429_2018,-0.0235,0.0405,-0.0023,0.0011,-0.0113,2018-03-29,459
351996,93429,201804.0,460.0,CBOE,CBOE GLOBAL MARKETS INC,184500,12503M108,523210,40203040,0.175603,...,1.284230e+10,114.099998,93429_2018,0.0028,0.0114,0.0054,0.0014,0.0036,2018-04-30,460
351997,93429,201805.0,461.0,CBOE,CBOE GLOBAL MARKETS INC,184500,12503M108,523210,40203040,0.081776,...,1.201168e+10,106.779999,93429_2018,0.0265,0.0526,-0.0318,0.0014,0.0389,2018-05-31,461


In [16]:
dataset.to_csv('dataset_files/merged_df.csv', index=False)
#dataset.columns

In [17]:
# Keshav: Splitting the dataset into each respective period based on the in-sample / out-of-sample recomendation
TRAINING_SPLIT = 0.50
VALIDATE_SPLIT = 0.75

train, validate, test = \
                        np.split(dataset.sample(frac=1, random_state=301), 
                        [int(TRAINING_SPLIT*len(dataset)), int(VALIDATE_SPLIT*len(dataset))])

print("Training", train.shape)
print("Validating", validate.shape)
print("Testing", test.shape)

Training (175999, 99)
Validating (88000, 99)
Testing (88000, 99)


In [18]:
#Fama-Macbeth Regression (NOTE: only use training set for regression - 1980-2010)
month_groups = train.groupby('monthid')

cross_sect_dict = {}
for factor in portfolio_factors:
    cross_sect_dict[factor + '_beta'] = []
cross_sect_dict['xret'] = []

for month, group in month_groups:
    ret = group['ret_f1']
    #ret = group['RET'] - group['RF']
    explanatory_variates = group[portfolio_factors]
    w = sm.add_constant(explanatory_variates, has_constant='add')
    cross_sect_reg = sm.OLS(ret, w, missing='drop').fit()
    for i in range(len(portfolio_factors)):
        #Note: params[0] is the alpha estimate
        beta_val = cross_sect_reg.params[i+1]
        cross_sect_dict[portfolio_factors[i] + '_beta'].append(beta_val)
    cross_sect_dict['xret'].append(ret.mean())

cross_sect_df = pd.DataFrame(cross_sect_dict)
cross_sect_df

,lag_log_size_beta,roe_beta,O_SCORE_Q_beta,pe_ttm_beta,INVESTMENT_GROWTH_3Y_beta,RSI_20_beta,ni_g_q_beta,xret
0,0.000129,-0.012299,-0.006741,-0.006513,-0.002919,0.008157,0.002674,0.019526
1,0.002251,0.011631,0.001594,-0.002922,-0.010998,0.004049,-0.004073,-0.004673
2,-0.001078,0.017215,0.003130,-0.008741,0.000161,0.001638,-0.007165,-0.045161
3,0.000737,0.014983,-0.009272,0.008763,0.000091,-0.001346,0.006702,0.027200
4,0.000039,0.011451,-0.006607,-0.010940,0.006505,0.014186,-0.002474,-0.026187
...,...,...,...,...,...,...,...,...
426,-0.018223,-0.000238,0.007524,-0.011115,-0.008165,-0.011543,-0.006261,0.046188
427,-0.003953,0.006635,-0.008313,-0.001517,-0.001523,-0.004490,-0.008693,0.015810
428,0.002030,-0.003075,-0.004177,0.000952,-0.002295,-0.010932,-0.005188,0.031084
429,-0.006255,0.002008,-0.001914,0.004653,-0.004256,-0.014514,0.003802,0.030565


In [19]:
for factor in portfolio_factors:
    print('{} T-stat: '.format(factor), stats.ttest_1samp(cross_sect_df['{}_beta'.format(factor)], 0)[0])

lag_log_size T-stat:  -5.0238438035937305
roe T-stat:  1.4329695909169546
O_SCORE_Q T-stat:  -3.441540413132288
pe_ttm T-stat:  1.4723759649214814
INVESTMENT_GROWTH_3Y T-stat:  -1.2907971845949118
RSI_20 T-stat:  -1.2327603002315308
ni_g_q T-stat:  1.138820501870873


In [20]:
# macbeth_data_with_const = sm.add_constant(cross_sect_df.iloc[:, :10])
# modelData = sm.OLS(cross_sect_df['xret'], macbeth_data_with_const).fit()
# print('tvalues: \n', modelData.tvalues)

In [21]:
#print(macbeth_data_with_const)